In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import Video

# === CONFIG ===
A, D, F = 4, 4, 40
W, H = 100, 53.3
records = []

# === Ball movement ===
ball_x, ball_y = [30], [25]
for _ in range(F - 1):
    ball_x.append(np.clip(ball_x[-1] + 0.8 + np.random.normal(0, 0.1), 0, W))
    ball_y.append(np.clip(ball_y[-1] + 0.1 + np.random.normal(0, 0.1), 0, H))
for t in range(F):
    records.append({'step': t, 'player_id': -1, 'x': ball_x[t], 'y': ball_y[t], 'role': 'ball'})

# === Unique movement logic per player ===
def move_toward_ball(x, y, bx, by, style):
    if style == 'direct':
        dx = (bx - x) * 0.25 + np.random.normal(0, 0.1)
        dy = (by - y) * 0.25 + np.random.normal(0, 0.1)
    elif style == 'curve_left':
        dx = (bx - x) * 0.2 + np.random.normal(0, 0.1)
        dy = (by - y + 10) * 0.2 + np.random.normal(0, 0.1)
    elif style == 'diagonal':
        dx = (bx - x + 5) * 0.2 + np.random.normal(0, 0.1)
        dy = (by - y - 5) * 0.2 + np.random.normal(0, 0.1)
    elif style == 'delayed':
        dx = (bx - x) * 0.1 + np.random.normal(0, 0.1)
        dy = (by - y) * 0.1 + np.random.normal(0, 0.1)
    elif style == 'shadow':
        dx = (bx - x) * 0.15 + np.random.normal(0, 0.1)
        dy = (by - y + 5) * 0.15 + np.random.normal(0, 0.1)
    elif style == 'zone':
        dx = (bx - x) * 0.05 + np.random.normal(0, 0.05)
        dy = (by - y) * 0.05 + np.random.normal(0, 0.05)
    elif style == 'collapse':
        dx = (bx - x) * 0.3 + np.random.normal(0, 0.1)
        dy = (by - y) * 0.3 + np.random.normal(0, 0.1)
    else:
        dx = dy = 0
    return dx, dy

styles = {
    0: 'direct',
    1: 'curve_left',
    2: 'diagonal',
    3: 'delayed',
    4: 'collapse',
    5: 'shadow',
    6: 'zone',
    7: 'collapse'
}

for pid in range(A + D):
    role = 'attacker' if pid < A else 'defender'
    x, y = [np.random.uniform(10, 90)], [np.random.uniform(10, 43)]
    for t in range(1, F):
        dx, dy = move_toward_ball(x[-1], y[-1], ball_x[t], ball_y[t], styles[pid])
        x.append(np.clip(x[-1] + dx, 0, W))
        y.append(np.clip(y[-1] + dy, 0, H))
    for t in range(F):
        records.append({'step': t, 'player_id': pid, 'x': x[t], 'y': y[t], 'role': role})

# === Save to submission.parquet ===
df = pd.DataFrame(records)
df.to_parquet("submission.parquet", index=False, engine="pyarrow", compression=None)
print("✅ submission.parquet created:", df.shape)

# === Animate players only ===
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_xlim(0, W)
ax.set_ylim(0, H)
ax.set_facecolor('darkgreen')
ax.set_title("4v4 Tactical Movement (Individual Logic)", fontsize=14)
ax.grid(True, color='white', linestyle='--', linewidth=0.5)

attackers_plot, = ax.plot([], [], 'b^', label='Attackers')
defenders_plot, = ax.plot([], [], 'rs', label='Defenders')
ball_plot, = ax.plot([], [], 'yo', label='Ball')
ax.legend()

def init():
    attackers_plot.set_data([], [])
    defenders_plot.set_data([], [])
    ball_plot.set_data([], [])
    return attackers_plot, defenders_plot, ball_plot

def update(frame):
    attackers = df[(df.role == 'attacker') & (df.step == frame)]
    defenders = df[(df.role == 'defender') & (df.step == frame)]
    ball = df[(df.role == 'ball') & (df.step == frame)]
    attackers_plot.set_data(attackers.x, attackers.y)
    defenders_plot.set_data(defenders.x, defenders.y)
    ball_plot.set_data(ball.x, ball.y)
    return attackers_plot, defenders_plot, ball_plot

ani = animation.FuncAnimation(fig, update, frames=F, init_func=init, blit=True, interval=200)
ani.save("individual_tactical_animation.mp4", writer="ffmpeg", fps=10)
print("🎥 Video saved as individual_tactical_animation.mp4")

Video("individual_tactical_animation.mp4", embed=True)